# PDE Model verification
## Pattern formation

## Objectives 
* Compare simulated diffusion from a point source to analytical solution in order to
    * measure convergence of the model 
    * as a way to measure accuracy of the numerical analysis
* Currently, the approach is just to use the solvers included with scipy along with a custom class to help with efficient jacobian calculation.
    * in the future, you will use a similar notebook and test code to verify your numerical experiments

## Model considerations 
* You've chosen the fitzhugh-nagumo model for verification because it can allow for diffusion-only or pattern formation solutions that are all well-studied analytically. this makes verification easier.
* equation
$$ \dot{v} = \mathrm{d}x Dv \nabla v - w + pn v - tv \frac{v^3}{3} $$
$$ \dot{w} = a(bv - cw) $$
* Species
    1. v, "potential"
    1. w, "recovery" 

* Reactions 
    1. `v - 1/3 * v^3` nonlinear term 
    1. Diffusion 
        * 2nd order central difference laplace approximation, 5-point "+" shape stencil
    1. Parameters
        * dx: Length modification of diffusion terms. In the compartmental model, diffusion is calculated via Ficks' first law, where the flux between two adjacent compartments is equal to the flux multiplied by the area of the interface between the components :  
        $\frac{\mathrm{d} C}{\mathrm{d} t} $ 
        in continuous form gives up 
        $\Delta C = D \frac{A}{V} \frac{\Delta C}{\Delta x} = D \frac{2.25 \cdot 5 \cdot \mathrm{scale}^2 \mathrm{mm}^2}{\mathrm{scale} \cdot 2.25^2 \cdot 5 \mathrm{mm}^3} \frac{\Delta C \cdot \mathrm{scale}}{2.25 \mathrm{mm}} = \frac{D \Delta C \mathrm{scale}^2}{2.25^2 \mathrm{mm}^2}$. the dx parameter below is the symbol $A$ in this equation. <- double check this
        * dx ^
        * Dv: diffusion coefficient (probably leave it 1. w does not diffuse)
        * bias: bias towards firing
        * pn: coefficient of v in $\dot{v}$
        * tv: coefficient of $V^3$ in $\dot{v}$ 
        * a,b,c: coefficients for the linear w equation

## 2D Discrete Laplacian

In continuous form : 
$$ U_t = \triangle U - \lambda U $$

In discrete form, for point $i$ : 
$$ \Delta U_i = \sum_{1 = w(i,j)}\omega(i,j)(U_i - U_j) - \lambda U_i $$

Use discrete laplacian approximation w/o diagonals for grid spacing, so that we can have zero-flux  boundary conditions. 

$$ L = 
 \begin{pmatrix}
  0 & 1 & 0 \\
  1 & -4 & 1 \\
  0 & 1 & 0 
 \end{pmatrix} $$

I apply this stencil as separate matrix sums for the domain interior, edges, and corners. 

In [ ]:
# imports
import numpy as np
import matplotlib as mpl
mpl.use("Agg")
import matplotlib.pyplot as plt 
plt.rcParams['animation.ffmpeg_path'] = '/usr/bin/ffmpeg' # Add the path of ffmpeg here!!
import matplotlib.animation as anm
from IPython.display import HTML
import omnisim as oms
import importlib
%matplotlib inline

In [ ]:
def write_movie(im_arr, t_vec, skip=1, n_frames=200):
    
    frames, s, h, w = im_arr.shape
    t_points = np.arange(0,t_vec.max(),n_frames)
    f_points = np.arange(frames)
    
    #frames = len(t)
    
    t, s, h, w = im_arr.shape
    xticks = []
    xticklabels = []
    # First set up the figure, the axis, and the plot element we want to animate
    blank_array = np.zeros([h, w])
    fig, axs = plt.subplots(1,2, figsize=(10,7))
    im_list = [0,0]
    
    # Plot cell densities
    ax = axs[0]
    indxs = [0]
    vmax = im_arr.max()
    vmin = im_arr.min()
    im = ax.imshow(blank_array, animated=True, vmax=vmax, vmin=vmin, interpolation='none', aspect=1)
    cbar = fig.colorbar(im, ax=ax, ticks=[vmin, vmax])
    ax.set_xticks([])
    ax.set_xticklabels(xticklabels)
    ax.set_yticks([])
    ax.set_title('potential')
    im_list[0] = im
    
    # Plot nutrient densities
    ax = axs[1]
    indxs = [1]
    vmax = im_arr.max()
    vmin = im_arr.min()
    im = ax.imshow(blank_array, animated=True, vmax=vmax, vmin=vmin, interpolation='none', aspect=1)
    cbar = fig.colorbar(im, ax=ax, ticks=[vmin, vmax])
    ax.set_xticks(xticks)
    ax.set_xticklabels(xticklabels)
    ax.set_yticks([])
    ax.set_title('recovery')
    im_list[1] = im

    # animation function.  This is called sequentially
    t_points = np.linspace(0,t_vec.max(), 200)
    f_inds = []
    t_ind = 0
    for tp in t_points:
        while tp > t_vec[t_ind]:
            t_ind += 1
        f_inds.append(t_ind-1)
        
    def animate(t_point):
        i = f_inds[t_point]
        
        # Plot cell densities
        ax = axs[0]
        indxs = [0]
        frame_arr = im_arr[i,indxs,:,:].sum(axis=0)
        im_list[0].set_array(frame_arr)

        # Plot nutrient densities
        ax = axs[1]
        indxs = [1]
        frame_arr = im_arr[i,indxs,:,:].sum(axis=0)
        im_list[1].set_array(frame_arr)

        #return im_list,

    # call the animator.  blit=True means only re-draw the parts that have changed.
    anim = anm.FuncAnimation(fig, animate, interval=50, frames=n_frames)

#     anim.save('./animation_test.gif', writer='pillow')
    fig.tight_layout()
    plt.close('all')
    return anim



In [ ]:
# oms = importlib.reload(oms)
simmer = oms.Simulator()
# simmer.basedims = np.array([64,64])
p0 = np.array([1e-1, 0, 4, 1, 0.1, 2, 1])
simmer.t_eval = np.linspace(0,100,200)
simmer.set_scale(5)
_, nh, nw, _ = simmer.dims
simmer.initial_array = simmer.initial_array*2 - 1 
simmer.sim(p0)
im_arr = simmer.sim_arr
t_vec = simmer.sim_tvc
anim = write_movie(im_arr, t_vec)
print('random initial')

In [ ]:
HTML(anim.to_html5_video())

In [ ]:
nt, ns, nh, nw = im_arr.shape
Dv = 1e-1
dx = simmer.dx
params = np.concatenate([[dx],p0])
y = im_arr[-10,:,:,]
dy = np.zeros_like(y)
diff_terms = np.zeros_like(y)
oms.calc_f(y, dy, diff_terms, params)
plt.imshow(dy[0])
plt.figure()
plt.imshow(np.log(np.abs(diff_terms[0]*dx*1e-1) / np.abs(dy[0])))

In [ ]:
np.mean(np.abs(dy[0]))

In [ ]:
init_arr = simmer.initial_array.copy()
init_arr = np.zeros_like(init_arr)
ns, nh, nw = init_arr.shape
init_arr[0,nh//2,nw//2] = 1
simmer.initial_array = init_arr
# simmer.t_eval = np.linspace(0,100,200)
# p0 = np.array([4e-2, 0, 1, 1, 0.8, 1, 0.08])
simmer.sim(p0)

im_arr = simmer.sim_arr
t_vec = simmer.sim_tvc
anim = write_movie(im_arr, t_vec)
# HTML(anim.to_html5_video())
print('spiked initial')

In [ ]:
HTML(anim.to_html5_video())